In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!git clone https://github.com/Anjaas85/authentic_vs_AI_generated_photos


Cloning into 'authentic_vs_AI_generated_photos'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 48 (delta 4), reused 11 (delta 0), pack-reused 30 (from 1)
Receiving objects: 100% (48/48), 161.00 MiB | 12.61 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
%cd authentic_vs_AI_generated_photos

/content/authentic_vs_AI_generated_photos


In [3]:
!ls


checkpoints  data  logs  models  two_class_classification.ipynb  utils


In [4]:
!git branch epochs20

In [5]:
!git checkout epochs20

Switched to branch 'epochs20'


In [6]:
# Downloading and Distributing the dataset
from torchvision.datasets import CIFAR10
from sklearn.model_selection import train_test_split
import os
from PIL import Image

# Step 1: Define paths
data_root = "./data"
train_dir = os.path.join(data_root, "train")
val_dir = os.path.join(data_root, "val")
class_dirs = ["class_0", "class_1"]

# Step 2: Create folder structure
for directory in [train_dir, val_dir]:
    for class_dir in class_dirs:
        os.makedirs(os.path.join(directory, class_dir), exist_ok=True)

# Step 3: Download CIFAR10 dataset
cifar10 = CIFAR10(root="./", download=True)
data, labels = cifar10.data, cifar10.targets

# Step 4: Simulate CIFAKE10 (first 5 classes as "authentic", next 5 as "AI-generated")
labels = [0 if label < 5 else 1 for label in labels]

# Step 5: Split data into train and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=labels
)

# Helper function to save images
def save_images(images, labels, save_dir):
    for idx, (image, label) in enumerate(zip(images, labels)):
        class_folder = os.path.join(save_dir, f"class_{label}")
        img = Image.fromarray(image)
        img.save(os.path.join(class_folder, f"{idx}.png"))

# Step 6: Save images to train and val folders
save_images(train_data, train_labels, train_dir)
save_images(val_data, val_labels, val_dir)

print("CIFAKE10 dataset distributed into train and validation folders.")




from models.densenet_model import create_densenet
from utils.data_loader import create_dataloaders, create_test_loader
from utils.evaluation import evaluate_model
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_recall_curve, f1_score
import numpy as np
import json
from sklearn.metrics import confusion_matrix, precision_recall_curve, f1_score
import numpy as np


# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paths and parameters
data_dir = 'data'

# Load train, validation, and test data
dataloaders = create_dataloaders(data_dir, batch_size=32)
test_loader = create_test_loader( batch_size=32)

# Load model
model = create_densenet(num_classes=2).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



# Training function with extended metrics saving
def train_model(model, criterion, optimizer, dataloaders, num_epochs=10):
    metrics_file_path = os.path.join('metrics', 'metrics_by_epoch.txt')
    os.makedirs('metrics', exist_ok=True)

    # Initialize storage for metrics
    metrics_data = {
        "precision_recall": [],
        "f1_scores": [],
        "val_probs": [],
        "val_labels": [],
        "confusion_matrices": [],
        "epoch_metrics": []
    }

    with open(metrics_file_path, 'w') as f:
        f.write(f'Epoch\tPhase\tLoss\tAccuracy\n')

        for epoch in range(num_epochs):
            print(f'Epoch {epoch+1}/{num_epochs}')
            print('-' * 10)

            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0
                all_labels = []
                all_probs = []

                for inputs, labels in dataloaders[phase]:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        probs = torch.softmax(outputs, dim=1)[:, 1]
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    all_labels.extend(labels.cpu().numpy().tolist())
                    all_probs.extend(probs.detach().cpu().numpy().tolist())

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                # Print and save metrics
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
                f.write(f'{epoch+1}\t{phase}\t{epoch_loss:.4f}\t{epoch_acc:.4f}\n')

                # Save metrics for validation phase
                if phase == 'val':
                    precision, recall, thresholds = precision_recall_curve(all_labels, all_probs)
                    f1 = f1_score(all_labels, np.round(all_probs))
                    confusion_mat = confusion_matrix(all_labels, np.round(all_probs))

                    metrics_data["precision_recall"].append({
                        "precision": precision.tolist(),
                        "recall": recall.tolist(),
                        "thresholds": thresholds.tolist(),
                    })
                    metrics_data["f1_scores"].append(float(f1))
                    metrics_data["val_probs"].append(all_probs)
                    metrics_data["val_labels"].append(all_labels)
                    metrics_data["confusion_matrices"].append(confusion_mat.tolist())

                    metrics_data["epoch_metrics"].append({
                        "epoch": epoch+1,
                        "loss": float(epoch_loss),
                        "accuracy": float(epoch_acc)
                    })

    # Save metrics data as JSON
    with open(os.path.join('metrics', 'metrics_data.json'), 'w') as json_file:
        json.dump(metrics_data, json_file, indent=4)

    print(f"Metrics saved in {metrics_file_path} and metrics_data.json")
    return model


# Train the model
trained_model = train_model(model, criterion, optimizer, dataloaders, num_epochs=20)



# Save the model
torch.save(trained_model.state_dict(), 'checkpoints/densenet_ai_vs_authentic.pth')
print("Model trained and saved!")

# Evaluate the model
print("Evaluating on validation set:")
evaluate_model(trained_model, dataloaders['val'], device,"validation")

print("Evaluating on test set:")
evaluate_model(trained_model, test_loader, device, "test")




100%|██████████| 170M/170M [00:13<00:00, 13.1MB/s]


Extracting ./cifar-10-python.tar.gz to ./
CIFAKE10 dataset distributed into train and validation folders.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 192MB/s]


...

train Loss: 0.4298 Acc: 0.8764
val Loss: 0.4388 Acc: 0.8688
Metrics saved in metrics/metrics_by_epoch.txt and metrics_data.json
Model trained and saved!
Evaluating on validation set:


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.88      0.87      5000
           1       0.87      0.86      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000

Confusion Matrix:
[[4378  622]
 [ 690 4310]]
Evaluating on test set:


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      5000
           1       0.87      0.86      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000

Confusion Matrix:
[[4362  638]
 [ 677 4323]]


In [ ]:
!git config --global user.email "anja.stanic2001@gmail.com"
!git config --global user.name "Anja Stanic"

In [ ]:
!git commit -m "First results for initial training on test and val"

In [ ]:
!git reset --soft HEAD~1


In [ ]:
!git reset

In [ ]:
!git log

In [ ]:
!git add new_main.ipynb

In [ ]:
!git add checkpoints/densenet_ai_vs_authentic.pth

In [ ]:
!git add utils/data_loader.py
!git add utils/evaluation.py
!git add models/densenet_model.py

In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   checkpoints/densenet_ai_vs_authentic.pth

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	cifar-10-batches-py/
	cifar-10-python.tar.gz
	data/train/class_0/
	data/train/class_1/
	data/val/class_0/
	data/val/class_1/
	models/__pycache__/
	utils/__pycache__/



In [ ]:
!git commit -m "Result of inital training for val and test"

[main 7d673d2d] Result of inital training for val and test
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 checkpoints/densenet_ai_vs_authentic.pth


In [ ]:
!